In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -U efficientnet --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from PIL import Image
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.utils import to_categorical
from efficientnet.tfkeras import preprocess_input
from keras.callbacks import EarlyStopping,ModelCheckpoint
from efficientnet.tfkeras import EfficientNetB3
from tensorflow.keras.models import load_model
from tensorflow.keras import datasets, layers, models
from keras.layers import Dropout, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D, Dense, Activation
from keras.optimizers import RMSprop, Adam

In [ ]:
!mkdir /kaggle/working/train
!mkdir /kaggle/working/val

In [ ]:
def resize_image(image, size=416):
    ''' Resizing the image to a square (416x416) '''
    w, h = image.size
    longest_side = max(w, h)
    # Scaling the image
    image = image.resize((int(w*size/longest_side), int(h*size/longest_side)), Image.BICUBIC) 
    scaled_w, scaled_h = image.size

    new_image = Image.new('RGB', (size, size))
    new_image.paste(image, ((size - scaled_w) // 2, (size - scaled_h) // 2))
    return new_image

In [ ]:
df = pd.read_csv('/kaggle/input/futurefish/species.csv')

train_data = []
val_data = []
class_names = [row[1] for row in df.itertuples()]

sets = ['train', 'val']
path = ['/kaggle/input/futurefish/training.csv' ,
        '/kaggle/input/futurefish/annotation.csv']

images_folder = '/kaggle/input/futurefish/data/data/'

for i in range(2):
    csv_path =  path[i]
    df = pd.read_csv(csv_path)
    for row in df.itertuples():
        file_id = row[1]
        species_id = row[2]
        new_filename = f'{file_id}.jpg'
        new_filepath = '/kaggle/working/{}/{}'.format(sets[i], new_filename)
        img = Image.open(f'{images_folder}{file_id}.jpg')
        img = img.convert('RGB')
        img = resize_image(img)
        img.save(new_filepath, format='JPEG', quality=90)
        
        set_data = train_data if i == 0 else val_data
        set_data.append((new_filepath,species_id))
        
        #if (row[0] % 100 == 0):
            #print(row[0])
       
    

len(train_data), len(val_data), len(class_names)

In [ ]:
class_names

In [ ]:
def load_set(set_data):
    X,y = [],[]
    for filepath, species_id in set_data:
        img = Image.open(filepath)
        img = np.array(img)
        X.append(img)
        y.append(species_id)
    X = np.array(X)
    y = to_categorical(y)
    return X, y

X_train, y_train = load_set(train_data)
X_val, y_val = load_set(val_data)


X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)

In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
filepath1 = "model_cnn.h5"
ckpt1 = ModelCheckpoint(filepath1, monitor='loss', verbose=1, save_best_only=True, mode='min')

model = models.Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(300, 300, 3)))
model.add(Convolution2D(4, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(4, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(8, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(8, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(20, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
        
history = model.fit(X_train, y_train, callbacks=[es,ckpt1], batch_size=16, epochs=200, validation_data=(X_val, y_val))

In [ ]:
model = load_model('model_cnn.h5')
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Model evaluation
test_loss, test_acc = model.evaluate(X_val, y_val, verbose=2)

In [ ]:
i = 40
plt.imshow((X_val[i]+1.)/2.)
model.predict(np.array([X_val[i]]))

In [ ]:
def test_video(path):
    cap = cv2.VideoCapture(path)
    success,image = cap.read()
    if not success:
        print('Could not open file or read frame')
    else:
        images = []
        i = 0
        while success:
            img = Image.fromarray(image)
            img = resize_image(img)
            images.append(np.array(img))
            
            cap.set(cv2.CAP_PROP_POS_MSEC,i*500)
            success,image = cap.read()
            i += 1
            if i > 15:
                break
        
        images = np.array(images)
        images = preprocess_input(images)
        predictions = model.predict(images)
        argmaxes = np.argmax(predictions, axis=-1)
        predictions = predictions*100
        print(argmaxes.shape)
        
        for i, img in enumerate(images):
            plt.figure(figsize=(7,7))
            plt.imshow(img/5.5+.5)
            plt.axis('off')
            plt.text(50, 50, 'Class:{} {:.2f}%'.format(class_names[argmaxes[i]], predictions[i][argmaxes[i]]), fontsize=15,color='White')
    

test_video('../input/test-video-fish/test2.mp4')